# 04 – Transformación: videos_snapshot

Este notebook transforma el JSON crudo del recurso `videos`
en una tabla de snapshot mensual por video, que permite
analizar métricas de rendimiento (vistas, likes, comentarios)
y su evolución mes a mes.

In [ ]:
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

RAW_PATH = PROJECT_ROOT / "data" / "raw" / "youtube"
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

PROCESSED_PATH.mkdir(parents=True, exist_ok=True)

RAW_PATH, PROCESSED_PATH

In [ ]:
import json
import pandas as pd
from datetime import datetime, date
import isodate

In [ ]:
with open(RAW_PATH / "videos.json", "r", encoding="utf-8") as f:
    videos_raw = json.load(f)

len(videos_raw["items"])


In [ ]:
from datetime import datetime, date

EXTRACTED_AT = datetime.utcnow()

EXTRACTED_AT

In [ ]:
import isodate

def iso_duration_to_seconds(duration):
    try:
        return int(isodate.parse_duration(duration).total_seconds())
    except Exception:
        return None


In [ ]:
from datetime import date, datetime, timezone

# Fecha de snapshot (solo fecha, sin hora)
SNAPSHOT_DATE = date.today()

# Timestamp explícitamente en UTC (timezone-aware)
EXTRACTED_AT = datetime.now(timezone.utc)

SNAPSHOT_DATE, EXTRACTED_AT

In [ ]:
import pandas as pd

rows = []

for item in videos_raw["items"]:
    rows.append({
        "snapshot_date": SNAPSHOT_DATE,
        "video_id": item["id"],
        "channel_id": item["snippet"]["channelId"],
        "published_at": pd.to_datetime(
            item["snippet"]["publishedAt"],
            utc=True
        ),
        "duration_seconds": iso_duration_to_seconds(
            item["contentDetails"].get("duration")
        ),
        "view_count": int(item["statistics"].get("viewCount", 0)),
        "like_count": int(item["statistics"].get("likeCount", 0)),
        "comment_count": int(item["statistics"].get("commentCount", 0)),
        "extracted_at": EXTRACTED_AT
    })

df_videos_snapshot = pd.DataFrame(rows)
df_videos_snapshot.head()



In [ ]:
df_videos_snapshot.shape

In [ ]:
df_videos_snapshot.dtypes

In [ ]:
type(df_videos_snapshot["snapshot_date"].iloc[0])

In [ ]:
#Ordenar columnas
df_videos_snapshot = df_videos_snapshot[
    [
        "snapshot_date",
        "video_id",
        "channel_id",
        "published_at",
        "duration_seconds",
        "view_count",
        "like_count",
        "comment_count",
        "extracted_at"
    ]
]

> ⚠️ Nota (fase de desarrollo)
>
> El guardado en formato Parquet se utiliza temporalmente para pruebas y separación entre notebooks.
> En la versión final del pipeline (.py), este paso será omitido y el DataFrame se enviará directamente a BigQuery, donde residirá el histórico definitivo.

In [ ]:
output_file = PROCESSED_PATH / "videos_snapshot.parquet"

df_videos_snapshot.to_parquet(output_file, index=False)

output_file



In [ ]:
import pandas as pd
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
file_path = PROJECT_ROOT / "data" / "processed" / "youtube" / "videos_snapshot.parquet"

df = pd.read_parquet(file_path)
df